## 1. Import Required Libraries

Import all necessary libraries for YOLO dog detection, computer vision, and data visualization.

In [11]:
# Core Libraries
import os
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# YOLO and Deep Learning
from ultralytics import YOLO
import torch

# Computer Vision
import cv2

# Data Science and Visualization
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# FiftyOne for Dataset Management

# Set FiftyOne database URI (adjust as needed)
# This must be set before importing FiftyOne
os.environ["FIFTYONE_DATABASE_URI"] = "mongodb://127.0.0.1:27017"

import fiftyone as fo
import fiftyone.zoo as foz
from fiftyone import ViewField as F

# Utilities
from tqdm import tqdm

# Check if GPU is available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")
if device == 'cuda':
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"CUDA Version: {torch.version.cuda}")

# Set matplotlib style
plt.style.use('default')
sns.set_palette("husl")

print(f"FiftyOne version: {fo.__version__}")

Using device: cuda
GPU: NVIDIA GeForce RTX 3060
CUDA Version: 12.8
FiftyOne version: 1.10.0


## 2. Setup Project Directories and Configuration

Create necessary directories and define project configuration for COCO dataset usage.

In [12]:
# Project Configuration
PROJECT_ROOT = Path.cwd()
DATA_DIR = PROJECT_ROOT / 'data'
IMAGES_DIR = DATA_DIR / 'images'
COCO_IMAGES_DIR = IMAGES_DIR / 'coco_dogs'
MODELS_DIR = PROJECT_ROOT / 'models'
OUTPUTS_DIR = PROJECT_ROOT / 'outputs'

# Create directories if they don't exist
for directory in [DATA_DIR, IMAGES_DIR, COCO_IMAGES_DIR, MODELS_DIR, OUTPUTS_DIR]:
    directory.mkdir(parents=True, exist_ok=True)

# Configuration dictionary - Pre-trained model settings
config = {
    'model_name': 'yolov8n.pt',  # Pre-trained YOLOv8 nano model
    'img_size': 640,
    'confidence_threshold': 0.25,
    'iou_threshold': 0.45,
    'dog_class_id': 16,  # COCO dataset dog class ID
    
    # FiftyOne-specific settings
    'fiftyone_dataset_name': 'coco-dogs-yolo',
    'fiftyone_port': 5151,  # Default FiftyOne App port
    'eval_key': 'yolo_eval',  # Key for storing evaluation results
}

print("Project Configuration:")
print(f"├── Data Directory: {DATA_DIR}")
print(f"├── COCO Images Directory: {COCO_IMAGES_DIR}")
print(f"├── Models Directory: {MODELS_DIR}")
print(f"└── Outputs Directory: {OUTPUTS_DIR}")
print(f"\nModel Configuration:")
for key, value in config.items():
    print(f"  {key}: {value}")

Project Configuration:
├── Data Directory: /home/mateusbarbosa/Documents/univali/inteligencia_artificial-ii/yolo-dog/data
├── COCO Images Directory: /home/mateusbarbosa/Documents/univali/inteligencia_artificial-ii/yolo-dog/data/images/coco_dogs
├── Models Directory: /home/mateusbarbosa/Documents/univali/inteligencia_artificial-ii/yolo-dog/models
└── Outputs Directory: /home/mateusbarbosa/Documents/univali/inteligencia_artificial-ii/yolo-dog/outputs

Model Configuration:
  model_name: yolov8n.pt
  img_size: 640
  confidence_threshold: 0.25
  iou_threshold: 0.45
  dog_class_id: 16
  fiftyone_dataset_name: coco-dogs-yolo
  fiftyone_port: 5151
  eval_key: yolo_eval


## 3. Define Helper Functions

Create utility functions for image processing, visualization, and dog detection.

In [13]:
def visualize_detections(image_path, results, save_path=None):
    """Visualize detection results with bounding boxes (matplotlib fallback)"""
    # Read image
    img = cv2.imread(str(image_path))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Plot results
    fig, ax = plt.subplots(1, 1, figsize=(12, 8))
    ax.imshow(img)
    
    # Draw bounding boxes
    if results and len(results[0].boxes) > 0:
        for box in results[0].boxes:
            # Get box coordinates
            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
            confidence = box.conf[0].cpu().numpy()
            class_id = int(box.cls[0].cpu().numpy())
            
            # Draw rectangle
            rect = plt.Rectangle((x1, y1), x2-x1, y2-y1, 
                                fill=False, edgecolor='red', linewidth=2)
            ax.add_patch(rect)
            
            # Add label
            label = f"Dog {confidence:.2f}"
            ax.text(x1, y1-10, label, color='red', fontsize=12,
                   bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    ax.axis('off')
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, bbox_inches='tight', dpi=150)
        print(f"Saved visualization to {save_path}")
    
    plt.show()
    return fig

def add_yolo_predictions_to_fiftyone(dataset, model, conf_threshold=0.25):
    """
    Run YOLO inference and add predictions to FiftyOne dataset
    
    Args:
        dataset: FiftyOne dataset
        model: YOLO model instance
        conf_threshold: Confidence threshold for detections
    
    Returns:
        Updated FiftyOne dataset with predictions
    """
    print(f"Running YOLO inference on {len(dataset)} samples...")
    
    with fo.ProgressBar() as pb:
        for sample in pb(dataset):
            # Run YOLO inference
            results = model(
                sample.filepath,
                conf=conf_threshold,
                classes=[16],  # Dog class in COCO
                verbose=False
            )
            
            # Convert YOLO results to FiftyOne Detections format
            detections = []
            
            if len(results[0].boxes) > 0:
                for box in results[0].boxes:
                    # Get box coordinates (YOLO format: xyxy)
                    x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                    confidence = float(box.conf[0].cpu().numpy())
                    
                    # Get image dimensions
                    height, width = results[0].orig_shape
                    
                    # Convert to FiftyOne format (relative coordinates: [x, y, w, h])
                    rel_box = [
                        x1 / width,           # top-left x
                        y1 / height,          # top-left y
                        (x2 - x1) / width,    # width
                        (y2 - y1) / height    # height
                    ]
                    
                    # Create FiftyOne Detection object
                    detection = fo.Detection(
                        label="dog",
                        bounding_box=rel_box,
                        confidence=confidence
                    )
                    detections.append(detection)
            
            # Add predictions to sample
            sample["yolo_predictions"] = fo.Detections(detections=detections)
            sample.save()
    
    print(f"Added YOLO predictions to {len(dataset)} samples")
    return dataset

def export_fiftyone_results_to_csv(dataset, output_path):
    """
    Export detection results from FiftyOne dataset to CSV
    
    Args:
        dataset: FiftyOne dataset with predictions
        output_path: Path to save CSV file
    
    Returns:
        DataFrame with results
    """
    results = []
    
    for sample in dataset:
        num_detections = 0
        avg_confidence = 0
        
        if sample.yolo_predictions is not None:
            dets = sample.yolo_predictions.detections
            num_detections = len(dets)
            if num_detections > 0:
                avg_confidence = sum(d.confidence for d in dets) / num_detections
        
        results.append({
            'image': Path(sample.filepath).name,
            'dogs_detected': num_detections,
            'confidence_avg': avg_confidence
        })
    
    df = pd.DataFrame(results)
    df.to_csv(output_path, index=False)
    print(f"Results exported to: {output_path}")
    
    return df

def process_video_detections(video_path, model, output_path, conf_threshold=0.25):
    """Process video file and detect dogs frame by frame"""
    cap = cv2.VideoCapture(str(video_path))
    
    # Get video properties
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Define codec and create VideoWriter
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(str(output_path), fourcc, fps, (width, height))
    
    print(f"Processing video: {total_frames} frames at {fps} FPS")
    
    frame_count = 0
    with tqdm(total=total_frames) as pbar:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            
            # Run detection
            results = model(frame, conf=conf_threshold, verbose=False)
            
            # Draw results on frame
            annotated_frame = results[0].plot()
            
            # Write frame
            out.write(annotated_frame)
            
            frame_count += 1
            pbar.update(1)
    
    cap.release()
    out.release()
    
    print(f"Processed {frame_count} frames. Output saved to {output_path}")
    return output_path

print("Helper functions defined successfully!")
print("  - visualize_detections() - Matplotlib visualization")
print("  - add_yolo_predictions_to_fiftyone() - Add YOLO predictions to FiftyOne")
print("  - export_fiftyone_results_to_csv() - Export FiftyOne results to CSV")
print("  - process_video_detections() - Process video files")

Helper functions defined successfully!
  - visualize_detections() - Matplotlib visualization
  - add_yolo_predictions_to_fiftyone() - Add YOLO predictions to FiftyOne
  - export_fiftyone_results_to_csv() - Export FiftyOne results to CSV
  - process_video_detections() - Process video files


## 4. Load Pre-trained YOLOv8 Model

Initialize YOLOv8 model with pre-trained COCO weights. The model already knows how to detect dogs (class 16 in COCO dataset).

In [14]:
# Load pre-trained YOLO model
print(f"Loading {config['model_name']} model into {MODELS_DIR}")
model = YOLO(config['model_name'])

# Display model information
print(f"\nModel loaded successfully!")
print(f"Model type: {type(model)}")
print(f"Device: {model.device}")

# Display COCO class names
class_names = model.names
print(f"\nTotal classes in COCO: {len(class_names)}")
print(f"Dog class ID: {config['dog_class_id']}")
print(f"Dog class name: {class_names[config['dog_class_id']]}")

# Show all animal classes in COCO
animal_classes = {k: v for k, v in class_names.items() 
                  if v in ['bird', 'cat', 'dog', 'horse', 'sheep', 
                          'cow', 'elephant', 'bear', 'zebra', 'giraffe']}
print(f"\nSome animal classes in COCO dataset:")
for class_id, class_name in sorted(animal_classes.items()):
    print(f"  {class_id}: {class_name}")

Loading yolov8n.pt model into /home/mateusbarbosa/Documents/univali/inteligencia_artificial-ii/yolo-dog/models

Model loaded successfully!
Model type: <class 'ultralytics.models.yolo.model.YOLO'>
Device: cpu

Total classes in COCO: 80
Dog class ID: 16
Dog class name: dog

Some animal classes in COCO dataset:
  14: bird
  15: cat
  16: dog
  17: horse
  18: sheep
  19: cow
  20: elephant
  21: bear
  22: zebra
  23: giraffe


## 5. Load COCO Dog Dataset with FiftyOne

Load COCO dataset using FiftyOne's built-in dataset zoo. FiftyOne automatically handles downloading, caching, and loading the dataset with proper annotations.

In [15]:
# =============================================================================
# Load COCO Dog Dataset with FiftyOne
# =============================================================================

print("="*60)
print("Loading COCO Dog Dataset with FiftyOne")
print("="*60)

dataset_name = config['fiftyone_dataset_name']

# Check if dataset already exists
if dataset_name in fo.list_datasets():
    print(f"Loading existing dataset: {dataset_name}")
    dataset = fo.load_dataset(dataset_name)
    print(f"Loaded {len(dataset)} samples from existing dataset")
else:
    print(f"Downloading and creating new dataset: {dataset_name}")
    print("This will download COCO validation images with dogs...")
    print("(FiftyOne handles all downloads and caching automatically)")
    
    # Load COCO dataset with dogs using FiftyOne
    # FiftyOne will download images and annotations automatically
    dataset = foz.load_zoo_dataset(
        "coco-2017",
        split="validation",
        label_types=["detections"],
        classes=["dog"],  # Only load images with dogs
        max_samples=50,
        dataset_name=dataset_name,
    )
    
    print(f"Loaded {len(dataset)} images with dogs")

print("\n" + "="*60)
print("Dataset Information")
print("="*60)
print(f"Dataset name: {dataset.name}")
print(f"Number of samples: {len(dataset)}")
print(f"Persistent: {dataset.persistent}")

# Show first few sample file paths
print("\nFirst 5 samples:")
for i, sample in enumerate(dataset.limit(5)):
    print(f"  {i+1}. {Path(sample.filepath).name}")

# Check if predictions already exist
if "yolo_predictions" in dataset.get_field_schema():
    print("\nYOLO predictions already exist in dataset")
    print("   (Skipping inference - predictions are already stored)")
    has_predictions = True
else:
    print("\nNo YOLO predictions found in dataset")
    print("   (Will run inference in next cell)")
    has_predictions = False

print("\n" + "="*60)
print("Ready for YOLO Inference")
print("="*60)

Loading COCO Dog Dataset with FiftyOne
Loading existing dataset: coco-dogs-yolo


ConnectionFailure: You have not defined a default connection

## 6. Run YOLO Inference and Add Predictions to FiftyOne

Run YOLOv8 inference on the dataset and store predictions in FiftyOne for interactive visualization and analysis.

In [ ]:
# =============================================================================
# Run YOLO Inference and Store Predictions in FiftyOne
# =============================================================================

print("="*60)
print("YOLO Inference on FiftyOne Dataset")
print("="*60)

# Check if predictions already exist
if "yolo_predictions" not in dataset.get_field_schema():
    print("Running YOLO inference on all samples...")
    print("This will add predictions to the FiftyOne dataset\n")
    
    # Add YOLO predictions to FiftyOne dataset
    dataset = add_yolo_predictions_to_fiftyone(
        dataset, 
        model, 
        conf_threshold=config['confidence_threshold']
    )
    
    print("\nYOLO predictions added successfully!")
else:
    print("YOLO predictions already exist in dataset")
    print("   (Reusing existing predictions)")

# =============================================================================
# Dataset Statistics and Analysis
# =============================================================================

print("\n" + "="*60)
print("Detection Statistics")
print("="*60)

# Count detections
samples_with_detections = len(dataset.match(F("yolo_predictions.detections").length() > 0))
total_detections = sum(
    len(s.yolo_predictions.detections) 
    for s in dataset 
    if s.yolo_predictions and s.yolo_predictions.detections
)

# Calculate average confidence
confidences = []
for sample in dataset:
    if sample.yolo_predictions and sample.yolo_predictions.detections:
        confidences.extend([d.confidence for d in sample.yolo_predictions.detections])

avg_confidence = sum(confidences) / len(confidences) if confidences else 0

print(f"Total samples: {len(dataset)}")
print(f"Samples with detections: {samples_with_detections}")
print(f"Samples without detections: {len(dataset) - samples_with_detections}")
print(f"Total dogs detected: {total_detections}")
print(f"Average confidence: {avg_confidence:.3f}")
print(f"Min confidence: {min(confidences):.3f}" if confidences else "N/A")
print(f"Max confidence: {max(confidences):.3f}" if confidences else "N/A")

# =============================================================================
# Show Sample Predictions
# =============================================================================

print("\n" + "="*60)
print("Sample Predictions (First 3 Images)")
print("="*60)

for i, sample in enumerate(dataset.limit(3)):
    print(f"\n{i+1}. {Path(sample.filepath).name}")
    
    if sample.yolo_predictions and sample.yolo_predictions.detections:
        num_dogs = len(sample.yolo_predictions.detections)
        print(f"   Detected {num_dogs} dog(s)")
        
        for j, detection in enumerate(sample.yolo_predictions.detections):
            print(f"     Dog {j+1}: Confidence={detection.confidence:.3f}")
    else:
        print(f"   No dogs detected")

print("\n" + "="*60)
print("Launch FiftyOne App for Interactive Visualization")
print("="*60)
print("\nTo launch the FiftyOne App in your browser, run:")
print("  session = fo.launch_app(dataset)")
print("\nOr to view specific samples:")
print("  session = fo.launch_app(dataset.limit(10))")
print("\nOr to filter high-confidence detections:")
print('  view = dataset.filter_labels("yolo_predictions", F("confidence") > 0.5)')
print("  session = fo.launch_app(view)")

YOLO Inference on FiftyOne Dataset
Running YOLO inference on all samples...
This will add predictions to the FiftyOne dataset

Running YOLO inference on 50 samples...
 100% |███████████████████| 50/50 [1.5s elapsed, 0s remaining, 51.8 samples/s]      
Added YOLO predictions to 50 samples

YOLO predictions added successfully!

Detection Statistics
Total samples: 50
Samples with detections: 36
Samples without detections: 14
Total dogs detected: 46
Average confidence: 0.730
Min confidence: 0.312
Max confidence: 0.939

Sample Predictions (First 3 Images)

1. 000000007386.jpg
   Detected 1 dog(s)
     Dog 1: Confidence=0.333

2. 000000017029.jpg
   Detected 1 dog(s)
     Dog 1: Confidence=0.367

3. 000000022192.jpg
   Detected 1 dog(s)
     Dog 1: Confidence=0.933

Launch FiftyOne App for Interactive Visualization

To launch the FiftyOne App in your browser, run:
  session = fo.launch_app(dataset)

Or to view specific samples:
  session = fo.launch_app(dataset.limit(10))

Or to filter high-

## 7. Launch FiftyOne App for Interactive Visualization

Launch the FiftyOne web application to interactively explore the dataset and predictions in your browser.

In [ ]:
# =============================================================================
# Launch FiftyOne App for Interactive Visualization
# =============================================================================

print("="*60)
print("Launching FiftyOne App")
print("="*60)

# Launch FiftyOne App with the dataset
# This will open a web browser with an interactive interface
print("\nStarting FiftyOne App...")
print("The app will open in your default web browser")
print(f"Default port: {config['fiftyone_port']}")

# Launch the app
session = fo.launch_app(
        dataset,
        port=config['fiftyone_port'],
        auto=False
    )
session.open_tab()

print("\nFiftyOne App is running!")
print(f"   URL: http://localhost:{config['fiftyone_port']}")
print("\nIn the FiftyOne App you can:")
print("  • Browse through all samples interactively")
print("  • Click on images to see detailed predictions")
print("  • Filter samples by confidence, number of detections, etc.")
print("  • Compare ground truth vs predictions side-by-side")
print("  • Tag samples for further analysis")
print("  • Export subsets of data")
print("\nUseful views to try:")
print('  • High confidence: dataset.filter_labels("yolo_predictions", F("confidence") > 0.7)')
print('  • Multiple dogs: dataset.match(F("yolo_predictions.detections").length() > 1)')
print('  • No detections: dataset.match(F("yolo_predictions.detections").length() == 0)')

# Keep session active
print("\nNote: The session will remain active while this notebook is running")

Launching FiftyOne App

Starting FiftyOne App...
The app will open in your default web browser
Default port: 5151


Session launched. Run `session.show()` to open the App in a cell output.


<IPython.core.display.Javascript object>


FiftyOne App is running!
   URL: http://localhost:5151

In the FiftyOne App you can:
  • Browse through all samples interactively
  • Click on images to see detailed predictions
  • Filter samples by confidence, number of detections, etc.
  • Compare ground truth vs predictions side-by-side
  • Tag samples for further analysis
  • Export subsets of data

Useful views to try:
  • High confidence: dataset.filter_labels("yolo_predictions", F("confidence") > 0.7)
  • Multiple dogs: dataset.match(F("yolo_predictions.detections").length() > 1)
  • No detections: dataset.match(F("yolo_predictions.detections").length() == 0)

Note: The session will remain active while this notebook is running


## 8. Export Results and Generate Reports

Export detection results to CSV and generate summary statistics from the FiftyOne dataset.

In [ ]:
# =============================================================================
# Export Results from FiftyOne to CSV
# =============================================================================

print("="*60)
print("Exporting Results")
print("="*60)

# Export results to CSV
results_csv = OUTPUTS_DIR / 'fiftyone_detection_results.csv'
results_df = export_fiftyone_results_to_csv(dataset, results_csv)

print("\n" + "="*60)
print("Results Summary")
print("="*60)
print(f"\nTotal images processed: {len(results_df)}")
print(f"Images with dogs detected: {(results_df['dogs_detected'] > 0).sum()}")
print(f"Images without detections: {(results_df['dogs_detected'] == 0).sum()}")
print(f"Total dogs detected: {results_df['dogs_detected'].sum()}")
print(f"Average dogs per image: {results_df['dogs_detected'].mean():.2f}")
print(f"Average confidence: {results_df[results_df['confidence_avg'] > 0]['confidence_avg'].mean():.3f}")

# Display top results
print("\n" + "="*60)
print("Top 10 Results by Confidence")
print("="*60)
top_results = results_df[results_df['dogs_detected'] > 0].nlargest(10, 'confidence_avg')
print(top_results.to_string(index=False))

# =============================================================================
# Advanced FiftyOne Queries and Views
# =============================================================================

print("\n" + "="*60)
print("Advanced Dataset Views")
print("="*60)

# Create useful views
high_conf_view = dataset.filter_labels("yolo_predictions", F("confidence") > 0.7)
multi_dog_view = dataset.match(F("yolo_predictions.detections").length() > 1)
no_detection_view = dataset.match(F("yolo_predictions.detections").length() == 0)

print(f"\nHigh confidence detections (>0.7): {len(high_conf_view)} samples")
print(f"Multiple dogs detected: {len(multi_dog_view)} samples")
print(f"No detections: {len(no_detection_view)} samples")

print("\nTo explore these views in the FiftyOne App:")
print("  session.view = high_conf_view  # Show only high confidence")
print("  session.view = multi_dog_view  # Show only multiple dogs")
print("  session.view = no_detection_view  # Show false negatives")

# =============================================================================
# Save Annotated Images (Optional)
# =============================================================================

print("\n" + "="*60)
print("Export Annotated Images (Optional)")
print("="*60)
print("\nTo save annotated images with bounding boxes:")
print("  for sample in dataset.limit(10):")
print("      # Use YOLO's built-in visualization")
print("      results = model(sample.filepath, conf=0.25)")
print("      annotated = results[0].plot()")
print("      output_path = OUTPUTS_DIR / f'annotated_{Path(sample.filepath).name}'")
print("      cv2.imwrite(str(output_path), annotated)")

# =============================================================================
# Dataset Evaluation Metrics (if ground truth available)
# =============================================================================

print("\n" + "="*60)
print("Model Evaluation Metrics")
print("="*60)

# Check if ground truth detections exist
if "ground_truth" in dataset.get_field_schema() or "detections" in dataset.get_field_schema():
    print("\nGround truth annotations detected!")
    print("You can evaluate model performance with:")
    print('  results = dataset.evaluate_detections(')
    print('      "yolo_predictions",')
    print('      gt_field="detections",  # or "ground_truth"')
    print('      eval_key="eval",')
    print('      compute_mAP=True,')
    print('  )')
    print('  print(f"mAP: {results.mAP()}")')
    print('  results.print_report()')
else:
    print("\nNo ground truth field found for evaluation")
    print("   The COCO dataset ground truth is stored in the 'detections' field")
    print("   You can evaluate against it to compare YOLO vs COCO annotations")

print("\n" + "="*60)
print("All Results Exported Successfully")
print("="*60)

Exporting Results
Results exported to: /home/mateusbarbosa/Documents/univali/inteligencia_artificial-ii/yolo-dog/outputs/fiftyone_detection_results.csv

Results Summary

Total images processed: 50
Images with dogs detected: 36
Images without detections: 14
Total dogs detected: 46
Average dogs per image: 0.92
Average confidence: 0.751

Top 10 Results by Confidence
           image  dogs_detected  confidence_avg
000000029393.jpg              1        0.938759
000000049269.jpg              1        0.933299
000000072813.jpg              1        0.933176
000000022192.jpg              1        0.932631
000000117525.jpg              1        0.928683
000000067213.jpg              1        0.924326
000000089880.jpg              2        0.917861
000000139872.jpg              1        0.913600
000000138492.jpg              1        0.900425
000000076417.jpg              1        0.887954

Advanced Dataset Views

High confidence detections (>0.7): 27 samples
Multiple dogs detected: 7 samples
N

## Summary

This notebook demonstrates how to use a pre-trained YOLOv8 model for dog detection on COCO dataset images, integrated with FiftyOne for powerful dataset management and visualization.

### Key Features
- **Pre-trained YOLOv8 model** - No training required, detects dogs immediately
- **FiftyOne Integration** - Automatic COCO dataset loading and management
- **Interactive Visualization** - Browser-based exploration of predictions via FiftyOne App
- **Dataset Persistence** - Predictions are stored and reusable across sessions
- **Advanced Filtering** - Query and filter samples by confidence, detections, etc.
- **Export & Analysis** - CSV export and statistical summaries

### FiftyOne Benefits
- **Automatic Downloads** - FiftyOne handles COCO dataset download and caching
- **Interactive Browser UI** - Explore predictions with clicks, not code
- **Persistent Storage** - Predictions saved to database, no re-computation
- **Powerful Queries** - Filter by confidence, bbox size, detection count, etc.
- **Model Comparison** - Compare multiple model predictions side-by-side
- **Evaluation Metrics** - Built-in mAP, precision, recall computation

### How to Use This Notebook
1. **Run all cells sequentially** - Dataset downloads only once, then reuses cached data
2. **Launch FiftyOne App** - Opens in browser for interactive exploration
3. **Filter and explore** - Use provided view examples to query the dataset
4. **Export results** - CSV files saved to `outputs/` directory

### FiftyOne Commands Reference

```python
# Load existing dataset
dataset = fo.load_dataset("coco-dogs-yolo")

# Launch app
session = fo.launch_app(dataset)

# Filter high confidence detections
high_conf = dataset.filter_labels("yolo_predictions", F("confidence") > 0.7)
session.view = high_conf

# Find images with multiple dogs
multi_dogs = dataset.match(F("yolo_predictions.detections").length() > 1)
session.view = multi_dogs

# Evaluate against ground truth (if available)
results = dataset.evaluate_detections(
    "yolo_predictions",
    gt_field="detections",
    eval_key="eval",
    compute_mAP=True
)
print(f"mAP: {results.mAP()}")
```

### Next Steps
- **Explore in FiftyOne App** - Click through samples interactively
- **Experiment with thresholds** - Adjust confidence values and re-run
- **Add video processing** - Use FiftyOne for frame-level video annotations
- **Compare models** - Add predictions from different YOLO versions
- **Train custom model** - See `yolo_dog_detection_custom.ipynb` for training

### Useful Resources
- [Ultralytics YOLO Documentation](https://docs.ultralytics.com/)
- [FiftyOne Documentation](https://docs.voxel51.com/)
- [FiftyOne Tutorials](https://docs.voxel51.com/tutorials/index.html)
- [COCO Dataset](https://cocodataset.org/)

---

**Project**: YOLO Dog Detection - COCO with FiftyOne  
**Date**: 2025  
**Model**: YOLOv8 Pre-trained  
**Dataset Management**: FiftyOne